In [117]:
import pandas as pd
from textwrap import dedent


ex_tables = {
    "別表:基本的臨床手技": {"path":"./raw/sheets/技能編集用/基本的臨床手技.csv"},
    "別表:基本的診療科": {"path":"./raw/sheets/技能編集用/基本的診療科.csv"},
}

output="<h1>表</h1>"

for table_name,info in ex_tables.items():
    table = pd.read_csv(info["path"])
    output += f"\n\n## {table_name}\n\n\n"+table.to_html()

table = pd.read_csv("./raw/sheets/知識編集用/別表-知識.csv")
table = table.loc[:,["臓器","分類","項目名"]]

def stack_samerows(table:pd.DataFrame,splitter:str=":::"):
    """ stack samerows in dataframe"""
    columns=list(table.columns.values)
    output_table=table.copy().fillna("")
    template_table=output_table.copy()
    for i in range(0,len(columns)-1):
        to_indexed = columns[0:i+1]
        indexed =template_table.groupby(to_indexed,as_index=False,sort=False)\
            .count()\
            .apply(lambda x:[x[i]+splitter+str(x[i+1]),*[None]*(x[i+1]-1)],axis=1)
        indexed = sum(indexed,[])
        output_table.iloc[:,i]=indexed
    return output_table

def make_html_table(table:pd.DataFrame,stack:bool=False):
    """ make html table from dataframe """
    SPLITTER="::-:-::"
    def to_table_cell(x):
        if x==None:
            return ""
        elif SPLITTER in x:
            s=x.split(SPLITTER)
            return f'<td rowspan="{s[1]}">{s[0]}</td>'
        else:
            return f"<td>{x}</td>"

    output_table= stack_samerows(table,SPLITTER) if stack else table.copy().fillna("")
    columns=list(table.columns.values)

    output_table=output_table.applymap(to_table_cell)

    theader=f"<thead><tr><th>{'</th><th>'.join(columns)}</th></thead></tr>"
    tbody="<tbody>\n"
    for items in output_table.fillna("").itertuples():
        row=f"<tr>{''.join(items[1:])}</tr>\n"
        tbody += row
    tbody+="</tbody>\n"
    table_html=f"<table border=1>{theader}\n{tbody}</table>"
    return table_html

output= make_html_table(table,stack=True)

with open("./dist/test_output.html","w") as f:
    f.write(output)

print("output... ./dist/test_output.html")


output... ./dist/test_output.html


In [133]:
import pandas as pd
from lib.dataframe_to_table import make_latex_table
from textwrap import dedent


table = pd.read_csv("./raw/sheets/知識編集用/別表-知識.csv")
table = table.loc[:,["臓器","分類","項目名"]]
output= make_latex_table(table,stack=True)

with open("./dist/test_output.tex","w") as f:
    f.write(output)

print("output... ./dist/test_output.tex")


output... ./dist/test_output.tex
